In [ ]:
#Mount the drive
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Import the required libraries.

import os
import glob
import keras
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

**Keras Sequence Video generators**

This package proposes some classes to work with Keras (included in TensorFlow) that generates batches of frames from video files.

In [ ]:
#Install the Video Generator to feed the videos to the network.
pip install keras-video-generators

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras_video import VideoFrameGenerator

In [ ]:
#Let's see the different classes we have
classes=os.listdir('/content/gdrive/MyDrive/Training Data')
print(classes)

['Anomaly', 'Normal']


In [ ]:
#Let's augment our frames according to our needs

#Specify the parameters according to your needs.
SIZE = (64, 64)
CHANNELS = 3
NBFRAME = 290
#Batch Size
BS = 4       


#Augment the frames as per your needs
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=False,
    rotation_range=8,
    width_shift_range=.2,     
    height_shift_range=.2)

In [ ]:
#The glob module is a useful part of the Python standard library. glob (short for global)
# is used to return all file paths that match a specific pattern

glob_pattern='/content/gdrive/MyDrive/Training Data/{classname}/*.mp4'

In [ ]:

#This is our train generator.
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=0.15, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

class Anomaly, validation count: 10, train count: 58
class Normal, validation count: 10, train count: 60
Total data: 2 classes for 118 files for train


In [ ]:
#Get the validation generator.
valid = train.get_validation_generator()

Total data: 2 classes for 20 files for validation


In [ ]:
#Let us define a model architecture.

model=Sequential()

model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (NBFRAME,
                                                                                      64, 64, 3)))
model.add(MaxPooling3D(pool_size=(1, 4, 4), padding='same', data_format='channels_last'))
# model.add(TimeDistributed(Dropout(0.2)))

model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
model.add(MaxPooling3D(pool_size=(1, 4, 4), padding='same', data_format='channels_last'))
# model.add(TimeDistributed(Dropout(0.2)))

model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
model.add(MaxPooling3D(pool_size=(1, 4, 4), padding='same', data_format='channels_last'))
# model.add(TimeDistributed(Dropout(0.2)))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
# model.add(Dense(256, activation = "softmax"))
model.add(Dense(2, activation = "sigmoid"))
model.summary()


# Create an Instance of Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'auto', restore_best_weights = True)

# Compile the model and specify loss function, optimizer and metrics values to the model
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 290, 62, 62, 4)    1024      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 290, 16, 16, 4)   0         
 )                                                               
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 290, 14, 14, 8)    3488      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 290, 4, 4, 8)     0         
 3D)                                                             
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 290, 2, 2, 14)     11144     
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 290, 1, 1, 14)    0

In [ ]:
#Let us train the model

model.fit(train, epochs=30,validation_data=valid,verbose=1,callbacks=[early_stopping_callback])

Epoch 1/30
29/29 [==============================] - 467s 16s/step - loss: 0.7514 - accuracy: 0.5517 - val_loss: 0.6849 - val_accuracy: 0.6500
Epoch 2/30
29/29 [==============================] - 233s 8s/step - loss: 0.6936 - accuracy: 0.5517 - val_loss: 0.6994 - val_accuracy: 0.5000
Epoch 3/30
29/29 [==============================] - 226s 8s/step - loss: 0.6805 - accuracy: 0.5603 - val_loss: 0.6513 - val_accuracy: 0.7000
Epoch 4/30
29/29 [==============================] - 226s 8s/step - loss: 0.6906 - accuracy: 0.5690 - val_loss: 0.6685 - val_accuracy: 0.5500
Epoch 5/30
29/29 [==============================] - 225s 8s/step - loss: 0.6780 - accuracy: 0.6207 - val_loss: 0.5073 - val_accuracy: 0.9000
Epoch 6/30
29/29 [==============================] - 225s 8s/step - loss: 0.5814 - accuracy: 0.6983 - val_loss: 0.7010 - val_accuracy: 0.6500
Epoch 7/30
29/29 [==============================] - 225s 8s/step - loss: 0.6110 - accuracy: 0.6638 - val_loss: 0.5406 - val_accuracy: 0.7000
Epoch 8/30
2

Since we are satisfied with the model, let's save it for future use.

In [ ]:
#Let us save the model.

model.save('/content/gdrive/MyDrive/Models./Model_10s_80_acc')

In [ ]:
train.classes

['Anomaly', 'Normal']

In [ ]:
pred=model.predict(valid)
pred

array([[0.9935812 , 0.00601557],
       [0.14265001, 0.83815205],
       [0.5526332 , 0.44180003],
       [0.6323789 , 0.37252486],
       [0.01798754, 0.98064774],
       [0.14081717, 0.86960065],
       [0.8836472 , 0.10627294],
       [0.13512555, 0.876844  ],
       [0.40590233, 0.6088207 ],
       [0.9941526 , 0.00473599],
       [0.9978941 , 0.00185127],
       [0.20801935, 0.8104378 ],
       [0.04771733, 0.94860834],
       [0.83949226, 0.16558394],
       [0.13364479, 0.87224895],
       [0.08451887, 0.9257098 ],
       [0.99396616, 0.00568637],
       [0.17632414, 0.84457546],
       [0.52089953, 0.46839073],
       [0.16164145, 0.8456164 ]], dtype=float32)